In [ ]:
# preamble
import pandas as pd
import math
import numpy as np


# Load data


In [ ]:
df = pd.read_csv("data/assets_with_deforestation.csv", sep = "\t")
df.head()

rename_aux = {str(x): "deforestation_"+str(x) for x in range(2000, 2023)}

df.rename(columns = rename_aux, inplace = True)

i_cols = [col for col in df.columns if not col.startswith('deforestation_')]

df = pd.wide_to_long(df, "deforestation_", i = i_cols, j = 'year').reset_index()
df['start_year_first'] = df.start_year_first.astype(int)
df.head(30) 
df.uid_gem.nunique()

In [ ]:
# df['y1'] = df.apply(lambda x: 1 if x.year in range(x.start_year_first - 0, x.start_year_first + 1) else 0, axis = 1)
# df['y3'] = df.apply(lambda x: 1 if x.year in range(x.start_year_first - 1, x.start_year_first + 2) else 0, axis = 1)
# df['y5'] = df.apply(lambda x: 1 if x.year in range(x.start_year_first - 2, x.start_year_first + 3) else 0, axis = 1)
# df['y7'] = df.apply(lambda x: 1 if x.year in range(x.start_year_first - 3, x.start_year_first + 4) else 0, axis = 1)

def year_var(n):
  return lambda x: 1 if x.year in range(x.start_year_first - math.floor(n/2), x.start_year_first + math.ceil(n/2)) else 0

for i in list(range(1, 10, 2)): 
  var = 'y' + str(i)
  df[var] = df.apply(year_var(i), axis = 1)

In [ ]:
df.head(30)

In [ ]:
# df['defo_y1'] = df.apply(lambda x: x.deforestation_ if x.y1 == 1 else 0, axis = 1)
# df['defo_y3'] = df.apply(lambda x: x.deforestation_ if x.y3 == 1 else 0, axis = 1)
# df['defo_y5'] = df.apply(lambda x: x.deforestation_ if x.y5 == 1 else 0, axis = 1)
# df['defo_y7'] = df.apply(lambda x: x.deforestation_ if x.y7 == 1 else 0, axis = 1)

def defo_var(n): 
    var = 'y' + str(n)
    return lambda x: x.deforestation_ if x[var] == 1 else 0

for i in list(range(1, 10, 2)): 
    var = 'defo_y' + str(i)
    df[var] = df.apply(defo_var(i), axis = 1)

In [ ]:
df.head(30)

In [ ]:
sum_cols = [x for x in df.columns if x not in i_cols and x != 'year'] + ['uid_gem']
sum_cols

df_group = df[sum_cols].groupby('uid_gem').sum().rename(columns = {'deforestation_': 'defo_total'}).reset_index()
df_group.uid_gem.nunique()

df_invariant = df[i_cols].groupby('uid_gem').agg('first').reset_index()
df_invariant.uid_gem.nunique()

df = pd.merge(df_invariant, df_group, how = 'inner', on = 'uid_gem')

df.uid_gem.nunique()



In [ ]:
# df['defo_y1'] = df.apply(lambda x: np.nan if x.y1 < 1 else x.defo_y1, axis = 1)
# df['defo_y3'] = df.apply(lambda x: np.nan if x.y3 < 3 else x.defo_y3, axis = 1)
# df['defo_y5'] = df.apply(lambda x: np.nan if x.y5 < 5 else x.defo_y5, axis = 1)
# df['defo_y7'] = df.apply(lambda x: np.nan if x.y7 < 7 else x.defo_y7, axis = 1)

def defo_fix(n): 
    var_yr = 'y' + str(n)
    var_defo = 'defo_' + var_yr
    return lambda x: np.nan if x[var_yr] < n else x[var_defo]

for i in list(range(1, 10, 2)): 
    var = 'defo_y' + str(i)
    df[var] = df.apply(defo_fix(i), axis = 1)

In [ ]:
df.head(30)
df.uid_gem.nunique()

In [ ]:
df.to_csv('data/regression_sample.csv', index=False, sep='\t', encoding='utf-8')